In [6]:
import os

# Set Spark version
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and extract Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xzf spark-3.5.0-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark

# Set environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Initialize findspark
import findspark
findspark.init()

In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(truncate=False)

+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [11]:
# 2. Create a temporary view of the DataFrame.
# use .createOrReplaceTempView("home_sales")
home_sales_df.createOrReplaceTempView("home_sales")

In [13]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
sql = """
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
sql = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [16]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
sql = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql).show()

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.8754439353942871 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales. - 'cache table home_sales'
spark.sql("cache table home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()


sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 0.7425563335418701 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data - 'p_home_sales'
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("p_home_sales")

In [23]:
# 11. Read the parquet formatted data. - spark.read.parquet('p_home_sales')
p_home_sales_df = spark.read.parquet('p_home_sales')

In [24]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView("p_home_sales")

In [27]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM p_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

--- 1.0113961696624756 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False